In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from env import *
from time import sleep
from prepare import *

pd.set_option("display.max_rows", None, "display.max_columns", None)

pd.reset_option("display.max_rows", "display.max_columns")

import re

In [2]:
names = pd.read_csv('more_players.csv')

In [3]:
names = names [['players']]

In [4]:
names

,players
0,Satoshi
1,Johnsun123
2,bogo binter
3,Sybr
4,good nightt
...,...
878,godsent mid
879,Alamander
880,hanason1
881,DabinPenguEmote


In [5]:
names[names.index==85]

,players
85,Lefasa


In [6]:
names

,players
0,Satoshi
1,Johnsun123
2,bogo binter
3,Sybr
4,good nightt
...,...
878,godsent mid
879,Alamander
880,hanason1
881,DabinPenguEmote


In [7]:
def get_puuid(name_list):
    puuid_list = []
    counter = 0
    for name in name_list:
        print(counter)
        counter += 1
        if counter%50 == 0:
            print('function paused')
            print
            time.sleep(180)
            print('Continue')
            response = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}?api_key={api_key}")
            summoner_info = response.json()
            puuid_list.append(summoner_info)
        else:
            response = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}?api_key={api_key}")
            summoner_info = response.json()
            puuid_list.append(summoner_info)
    return puuid_list

In [8]:
# puuid_list = get_puuid(names.players)

In [9]:
# puuid_list

In [10]:
# puuid_df = pd.DataFrame(puuid_list)

In [11]:
# puuid_df.drop(columns = 'status', inplace=True)

In [12]:
# puuid_df.dropna(inplace=True)

In [13]:
# puuid_df.to_csv('puuid.csv',index=False)

In [14]:
# puuid = puuid_df[['puuid']]

In [15]:
def get_match_info(match_ids, api_key, username, time = 20):
    """
    This function takes in a list of match ids and iterates through them. For each iteration,
    it will make two api calls and retrieve the necessary information for our project.
    
    Two json lists will be created, and once all the info has been gathered for each match id,
    this function will save both json lists for future reference. 
    
    This function also takes in a username string. This will be used for naming the saved files.
    
    Finally, this function will funnel the resulting json lists into a prepare function and
    return a complete df.
    
    The time parameter is used in the prepare function that will be called at the end
    and represents the timeframe of the data we want to gather and prepare.
    The default value is 20 for the 20 minute mark of the match.
    
    """
    
    #Create an empty list to store the timeline json data
    timeline_data = []
    
    #Create an empty list to store the other json data
    game_data = []
    
    #Set up main url
    url = 'https://americas.api.riotgames.com/'
    
    #Create a count var
    n = 0
    
    #Store the length of the match_ids list in a var
    match_count = len(match_ids)
    
    #Loop through each match_id
    for match in match_ids:
        #Update the count var
        n += 1
        
        #After every 50 match ids (100 total requests), wait 150 seconds (2.5 minutes)
        if n % 50 == 0:
            print(f'Completed {n} of {match_count} match IDs.')
            print(f'Waiting 150 seconds...\n')
            
            #Sleep 150 seconds
            sleep(150)
            
            print('Continuing!\n')
        
        ############################# TIMELINE DATA #######################
        #Set up timeline url
        timeline_query = f'lol/match/v5/matches/{match}/timeline/?api_key={api_key}'
        
        #Grab timeline json data
        timeline_response = requests.get(url + timeline_query)
        
        #Check response and leave a status message
        if timeline_response.status_code != 200:
            print(f'Something went wrong getting TIMELINE DATA! Status Code {game_response.status_code} for match ID: {match}.')
            print(f'Skipping this match ID.\n')
            continue

        #Turn it into json format
        timeline_json = timeline_response.json()
        
        #Append this data to the timeline_data list
        timeline_data.append(timeline_json)
        
        ############################# OTHER GAME DATA #######################
        #Set up game data url
        game_query = f'lol/match/v5/matches/{match}?api_key={api_key}'
        
        #Grab game json data
        game_response = requests.get(url + game_query)
        
        #Check response and leave a status message.
        if game_response.status_code != 200:
            print(f'Something went wrong getting OTHER GAME DATA! Status Code {game_response.status_code} for match ID: {match}.')
            print(f'Skipping this match ID and REMOVING PREVIOUS TIMELINE ENTRY.\n')
            
            #Remove the last entry in the timeline_data list
            timeline_data.pop()
            continue
        
        #Turn it into json format
        game_json = game_response.json()
        
        #Append this data to the game_data list
        game_data.append(game_json)
        
    ################################## END LOOP ##############################
    
    #Now that all of the json data has been gathered, save each file for future reference
    
    #Save timeline data.
    #First, convert it to a df
    timeline_df = pd.DataFrame(timeline_data)
    
    #Save as csv file. Use username string to identify whose file it is
    timeline_df.to_json(f'timeline_data_{username}.json')
    
    #Print status message
    print(f'Created timeline_data_{username}.json')
    
    #Save other game data
    #First, convert it to a df
    game_df = pd.DataFrame(game_data)
    
    #Save as csv file. Use username string to identify whose file it is
    game_df.to_json(f'other_game_data_{username}.json')
    
    #Print status message
    print(f'Created other_game_data_{username}.json\n')
    
    ################################ PREPARE DATA #########################
    
    #The following section will funnel the data into the prepare function
    #Written by Joshua C.
    
    #Leave a status message
    print('Preparing the data...\n')
    
    #Begin preparing the data
    df = prepare.prepare(timeline_data, game_data, time)
    
    #Finally, return the prepared df
    return df

In [16]:
match_ids = pd.read_csv('match_ids.csv')

In [17]:
match_ids = list(match_ids['0'])[1000:2000]

In [18]:
# df = get_match_info(match_ids, api_key, username, time=20)

In [19]:
other_game_data = pd.read_json('other_game_data_bryant.json')

In [20]:
timeline_data = pd.read_json('timeline_data_bryant.json')

In [21]:
timeline_list = timeline_data.to_dict(orient = 'records')

In [22]:
other_game_list = other_game_data.to_dict(orient = 'records')

In [23]:
# df = prepare(timeline_list, other_game_list, time=20)

In [24]:
# df.head()

In [25]:
# df.size

In [26]:
# df.shape

In [27]:
# df.columns

In [28]:
# df.to_csv('bryant_games.csv')

In [29]:
# df.winningTeam.value_counts()

In [30]:
#Load the match_ids.csv file
matches_df = pd.read_csv('match_ids.csv')
matches_df

,0
0,NA1_4062082172
1,NA1_4093908534
2,NA1_4094915650
3,NA1_4082128299
4,NA1_4083500908
...,...
37279,NA1_4097058147
37280,NA1_4096721229
37281,NA1_4094049450
37282,NA1_4091691873


In [31]:
#Set up the match id list. You only need to uncomment the line following your name

match_ids = list(matches_df['0'])[10_657:17_314]

In [32]:
#Set up your username. You only need to uncomment the line following your name.
#Again, this username is only for naming the json files.

#Joshua Bryant
username = 'bryant'

In [33]:
def get_match_info(match_ids, api_key, username, time = 20):
    """
    This function takes in a list of match ids and iterates through them. For each iteration,
    it will make two api calls and retrieve the necessary information for our project.
    
    Two json lists will be created, and once all the info has been gathered for each match id,
    this function will save both json lists for future reference. 
    
    This function also takes in a username string. This will be used for naming the saved files.
    
    Finally, this function will funnel the resulting json lists into a prepare function and
    return a complete df.
    
    The time parameter is used in the prepare function that will be called at the end
    and represents the timeframe of the data we want to gather and prepare.
    The default value is 20 for the 20 minute mark of the match.
    
    """
    
    #Create an empty list to store the timeline json data
    timeline_data = []
    
    #Create an empty list to store the other json data
    game_data = []
    
    #Set up main url
    url = 'https://americas.api.riotgames.com/'
    
    #Create a count var
    n = 0
    
    #Store the length of the match_ids list in a var
    match_count = len(match_ids)
    
    #Loop through each match_id
    for match in match_ids:
        #Update the count var
        n += 1
        
        #After every 50 match ids (100 total requests), wait 150 seconds (2.5 minutes)
        if n % 50 == 0:
            print(f'Completed {n} of {match_count} match IDs.')
            print(f'Waiting 150 seconds...\n')
            
            #Sleep 150 seconds
            sleep(150)
            
            print('Continuing!\n')
        
        ############################# TIMELINE DATA #######################
        #Set up timeline url
        timeline_query = f'lol/match/v5/matches/{match}/timeline/?api_key={api_key}'
        
        #Grab timeline json data
        timeline_response = requests.get(url + timeline_query)
        
        #Check response and leave a status message
        if timeline_response.status_code != 200:
            print(f'Something went wrong getting TIMELINE DATA! Status Code {timeline_response.status_code} for match ID: {match}.')
            print(f'Skipping this match ID.\n')
            continue

        #Turn it into json format
        timeline_json = timeline_response.json()
        
        #Append this data to the timeline_data list
        timeline_data.append(timeline_json)
        
        ############################# OTHER GAME DATA #######################
        #Set up game data url
        game_query = f'lol/match/v5/matches/{match}?api_key={api_key}'
        
        #Grab game json data
        game_response = requests.get(url + game_query)
        
        #Check response and leave a status message.
        if game_response.status_code != 200:
            print(f'Something went wrong getting OTHER GAME DATA! Status Code {game_response.status_code} for match ID: {match}.')
            print(f'Skipping this match ID and REMOVING PREVIOUS TIMELINE ENTRY.\n')
            
            #Remove the last entry in the timeline_data list
            timeline_data.pop()
            continue
        
        #Turn it into json format
        game_json = game_response.json()
        
        #Append this data to the game_data list
        game_data.append(game_json)
        
    ################################## END LOOP ##############################
    
    #Now that all of the json data has been gathered, save each file for future reference
    
    #Save timeline data.
    #First, convert it to a df
    timeline_df = pd.DataFrame(timeline_data)
    
    #Save as csv file. Use username string to identify whose file it is
    timeline_df.to_json(f'timeline_data_{username}.json')
    
    #Print status message
    print(f'Created timeline_data_{username}.json')
    
    #Save other game data
    #First, convert it to a df
    game_df = pd.DataFrame(game_data)
    
    #Save as csv file. Use username string to identify whose file it is
    game_df.to_json(f'other_game_data_{username}.json')
    
    #Print status message
    print(f'Created other_game_data_{username}.json\n')
    
    ################################ PREPARE DATA #########################
    
    #The following section will funnel the data into the prepare function
    #Written by Joshua C.
    
    #Leave a status message
    print('Preparing the data...\n')
    
    #Begin preparing the data
    df = prepare.prepare(timeline_data, game_data, time)
    
    #Finally, return the prepared df
    return df

In [34]:
#Call the get_match_info function
df = get_match_info(match_ids, api_key, username)

Completed 50 of 6657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 100 of 6657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 150 of 6657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 200 of 6657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 250 of 6657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 300 of 6657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 350 of 6657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 400 of 6657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 450 of 6657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 500 of 6657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 550 of 6657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 600 of 6657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 650 of 6657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 700 of 6657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 750 of 6657

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4102921581.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4080888024.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4083644157.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4085189550.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4078597362.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4094048704.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4074105858.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4101714233.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_40

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4081187157.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4085179036.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4077327601.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4103747251.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4101888728.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4076006613.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4100301533.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4094946535.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_40

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4059691742.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4067269791.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4071801834.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4091522347.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4086679527.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4046914462.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4103496909.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4100650865.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_41

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4095027786.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4073144036.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4096347966.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4099045355.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4075874156.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4067099371.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4090862665.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4071379696.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_40

...

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4053290109.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4082399642.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4092250030.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4064790642.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4082414810.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4088147654.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4076057500.
Skipping this match ID.

Completed 6400 of 6657 match IDs.
Waiting 150 seconds...

Continuing!

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4098444307.
Skipping this match ID.

Something

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4040002038.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4099256012.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4071042297.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4066613760.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4087416614.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4100856898.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4094125881.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4090832752.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_40

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4079251331.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4102293936.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4091689939.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4057793850.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4081628341.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4088755747.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4096188497.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4097986982.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_40

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4075020349.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4094309136.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4083595668.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4093768216.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4090593952.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4073940233.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4099505895.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_4097026472.
Skipping this match ID.

Something went wrong getting TIMELINE DATA! Status Code 403 for match ID: NA1_40

AttributeError: 'function' object has no attribute 'prepare'

In [35]:
df

NameError: name 'df' is not defined